<a href="https://colab.research.google.com/github/SomakSaha/Chronic-Kidney-Disease-CKD-detection-/blob/main/ckd_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:

df= pd.read_csv('/content/drive/MyDrive/Diabetis/kidney_disease.csv')
df.head()

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
df.drop("id", axis=1, inplace=True)


In [ ]:
df["classification"] = df["classification"].replace('ckd\t','ckd')

In [ ]:
df["classification"].unique()

array(['ckd', 'notckd'], dtype=object)

In [ ]:
df['pcv'] = pd.to_numeric(df['pcv'], errors='coerce')
df['wc'] = pd.to_numeric(df['wc'], errors='coerce')
df['rc'] = pd.to_numeric(df['rc'], errors='coerce')

In [ ]:
cat_cols = [col for col in df.columns if df[col].dtype == 'object']
num_cols = [col for col in df.columns if df[col].dtype != 'object']

In [ ]:
for col in cat_cols:
    print(f"{col} has {df[col].unique()} values\n")

rbc has [nan 'normal' 'abnormal'] values

pc has ['normal' 'abnormal' nan] values

pcc has ['notpresent' 'present' nan] values

ba has ['notpresent' 'present' nan] values

htn has ['yes' 'no' nan] values

dm has ['yes' 'no' ' yes' '\tno' '\tyes' nan] values

cad has ['no' 'yes' '\tno' nan] values

appet has ['good' 'poor' nan] values

pe has ['no' 'yes' nan] values

ane has ['no' 'yes' nan] values

classification has ['ckd' 'notckd'] values



In [ ]:
def random_value_imputation(feature):
    random_sample = df[feature].dropna().sample(df[feature].isna().sum())
    random_sample.index = df[df[feature].isnull()].index
    df.loc[df[feature].isnull(), feature] = random_sample

def impute_mode(feature):
    mode = df[feature].mode()[0]
    df[feature] = df[feature].fillna(mode)

In [ ]:
for col in num_cols:
    random_value_imputation(col)

In [ ]:
random_value_imputation('rbc')
random_value_imputation('pc')

for col in cat_cols:
    impute_mode(col)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in cat_cols:
    df[col] = le.fit_transform(df[col])

In [ ]:
ind_col = [col for col in df.columns if col != 'classification']
dep_col = 'classification'

x = df[ind_col]
y = df[dep_col]

In [ ]:
X=x
Y=y


In [ ]:
X = X.values
Y = Y.values


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
Y= pd.Series(Y)


In [ ]:
type(X)
type(Y)

pandas.core.series.Series

**Without SMOTE**

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score

# Assuming X and Y are your input and target data
# Define the number of folds
num_folds = 5

# Initialize lists to store the evaluation results
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
mcc_scores = []
auc_roc_scores = []



# Perform stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True)
for train_index, test_index in skf.split(X, Y):
    # Split the data into training and test sets for the current fold
    X_train, X_test = X[train_index], X[test_index]
    # X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    #start
    X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout

    model = Sequential()

# Add an LSTM layer with desired units and input shape
    model.add(LSTM(units=50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))  # Adding dropout for regularization

# Add another LSTM layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))

# Add a dense output layer for classification (adjust the number of units according to your task)
    model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train_reshaped, Y_train, epochs=15, batch_size=32, validation_data=(X_test_reshaped, Y_test))
    Y_pred = model.predict(X_test_reshaped)
    Y_pred_binary = np.round(Y_pred).flatten()

    # Calculate evaluation metrics
    accuracy = accuracy_score(Y_test, Y_pred_binary)
    precision = precision_score(Y_test, Y_pred_binary)
    recall = recall_score(Y_test, Y_pred_binary)
    f1 = f1_score(Y_test, Y_pred_binary)
    mcc = matthews_corrcoef(Y_test, Y_pred_binary)
    auc_roc = roc_auc_score(Y_test, Y_pred_binary)

    # Append the scores to the respective lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    mcc_scores.append(mcc)
    auc_roc_scores.append(auc_roc)

# Calculate the average scores
avg_accuracy = np.mean(accuracy_scores)
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_f1 = np.mean(f1_scores)
avg_mcc = np.mean(mcc_scores)
avg_auc_roc = np.mean(auc_roc_scores)

# Print the average scores
print("Average Test Accuracy:", avg_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1 Score:", avg_f1)
print("Average MCC:", avg_mcc)
print("Average AUC-ROC:", avg_auc_roc)



Epoch 1/15
10/10 [==============================] - 5s 126ms/step - loss: 0.6814 - accuracy: 0.7594 - val_loss: 0.6648 - val_accuracy: 0.9250
Epoch 2/15
10/10 [==============================] - 0s 10ms/step - loss: 0.6508 - accuracy: 0.9438 - val_loss: 0.6302 - val_accuracy: 0.9500
Epoch 3/15
10/10 [==============================] - 0s 13ms/step - loss: 0.6121 - accuracy: 0.9344 - val_loss: 0.5808 - val_accuracy: 0.9375
Epoch 4/15
10/10 [==============================] - 0s 16ms/step - loss: 0.5541 - accuracy: 0.9250 - val_loss: 0.5130 - val_accuracy: 0.8875
Epoch 5/15
10/10 [==============================] - 0s 14ms/step - loss: 0.4852 - accuracy: 0.9219 - val_loss: 0.4346 - val_accuracy: 0.8875
Epoch 6/15
10/10 [==============================] - 0s 14ms/step - loss: 0.4038 - accuracy: 0.9156 - val_loss: 0.3589 - val_accuracy: 0.8875
Epoch 7/15
10/10 [==============================] - 0s 15ms/step - loss: 0.3297 - accuracy: 0.9219 - val_loss: 0.2981 - val_accuracy: 0.9125
Epoch 8/15
1

**SMOTE APPLYING**

In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE

# Assuming you already have your DataFrame and column setup as described
# X is the feature DataFrame, and y is the target Series

# Initialize the SMOTE object
smote = SMOTE(random_state=42)


# Apply SMOTE oversampling to the data
X_resampled, y_resampled = smote.fit_resample(x, y)


In [ ]:
X=X_resampled

Y=y_resampled

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X=X_scaled

In [ ]:
print((Y[Y==0].count()/Y.count())*100)
print((Y[Y==1].count()/Y.count())*100)
print(Y[Y==1].count()+Y[Y==1].count())

50.0
50.0
500


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


**Adam with LR=0.001**

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score

# Assuming X and Y are your input and target data
# Define the number of folds
num_folds = 5

# Initialize lists to store the evaluation results
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
mcc_scores = []
auc_roc_scores = []



# Perform stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True)
for train_index, test_index in skf.split(X, Y):
    # Split the data into training and test sets for the current fold
    X_train, X_test = X[train_index], X[test_index]
    # X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    #start
    X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout

    model = Sequential()

# Add an LSTM layer with desired units and input shape
    model.add(LSTM(units=50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))  # Adding dropout for regularization

# Add another LSTM layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))

# Add a dense output layer for classification (adjust the number of units according to your task)
    model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train_reshaped, Y_train, epochs=15, batch_size=32, validation_data=(X_test_reshaped, Y_test))
    Y_pred = model.predict(X_test_reshaped)
    Y_pred_binary = np.round(Y_pred).flatten()

    # Calculate evaluation metrics
    accuracy = accuracy_score(Y_test, Y_pred_binary)
    precision = precision_score(Y_test, Y_pred_binary)
    recall = recall_score(Y_test, Y_pred_binary)
    f1 = f1_score(Y_test, Y_pred_binary)
    mcc = matthews_corrcoef(Y_test, Y_pred_binary)
    auc_roc = roc_auc_score(Y_test, Y_pred_binary)

    # Append the scores to the respective lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    mcc_scores.append(mcc)
    auc_roc_scores.append(auc_roc)

# Calculate the average scores
avg_accuracy = np.mean(accuracy_scores)
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_f1 = np.mean(f1_scores)
avg_mcc = np.mean(mcc_scores)
avg_auc_roc = np.mean(auc_roc_scores)

# Print the average scores
print("Average Test Accuracy:", avg_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1 Score:", avg_f1)
print("Average MCC:", avg_mcc)
print("Average AUC-ROC:", avg_auc_roc)



Epoch 1/15
13/13 [==============================] - 7s 148ms/step - loss: 0.6803 - accuracy: 0.7650 - val_loss: 0.6587 - val_accuracy: 0.9300
Epoch 2/15
13/13 [==============================] - 0s 16ms/step - loss: 0.6373 - accuracy: 0.9350 - val_loss: 0.6029 - val_accuracy: 0.9400
Epoch 3/15
13/13 [==============================] - 0s 14ms/step - loss: 0.5691 - accuracy: 0.9375 - val_loss: 0.5113 - val_accuracy: 0.9500
Epoch 4/15
13/13 [==============================] - 0s 15ms/step - loss: 0.4648 - accuracy: 0.9375 - val_loss: 0.3957 - val_accuracy: 0.9500
Epoch 5/15
13/13 [==============================] - 0s 11ms/step - loss: 0.3569 - accuracy: 0.9350 - val_loss: 0.2894 - val_accuracy: 0.9500
Epoch 6/15
13/13 [==============================] - 0s 11ms/step - loss: 0.2619 - accuracy: 0.9350 - val_loss: 0.2181 - val_accuracy: 0.9500
Epoch 7/15
13/13 [==============================] - 0s 10ms/step - loss: 0.2035 - accuracy: 0.9375 - val_loss: 0.1770 - val_accuracy: 0.9500
Epoch 8/15
1

**LR=0.0001**

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score

# Assuming X and Y are your input and target data
# Define the number of folds
num_folds = 5

# Initialize lists to store the evaluation results
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
mcc_scores = []
auc_roc_scores = []



# Perform stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True)
for train_index, test_index in skf.split(X, Y):
    # Split the data into training and test sets for the current fold
    X_train, X_test = X[train_index], X[test_index]
    # X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    #start
    X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout

    model = Sequential()

# Add an LSTM layer with desired units and input shape
    model.add(LSTM(units=50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))  # Adding dropout for regularization

# Add another LSTM layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))

# Add a dense output layer for classification (adjust the number of units according to your task)
    model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
    custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train_reshaped, Y_train, epochs=15, batch_size=32, validation_data=(X_test_reshaped, Y_test))
    Y_pred = model.predict(X_test_reshaped)
    Y_pred_binary = np.round(Y_pred).flatten()

    # Calculate evaluation metrics
    accuracy = accuracy_score(Y_test, Y_pred_binary)
    precision = precision_score(Y_test, Y_pred_binary)
    recall = recall_score(Y_test, Y_pred_binary)
    f1 = f1_score(Y_test, Y_pred_binary)
    mcc = matthews_corrcoef(Y_test, Y_pred_binary)
    auc_roc = roc_auc_score(Y_test, Y_pred_binary)

    # Append the scores to the respective lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    mcc_scores.append(mcc)
    auc_roc_scores.append(auc_roc)

# Calculate the average scores
avg_accuracy = np.mean(accuracy_scores)
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_f1 = np.mean(f1_scores)
avg_mcc = np.mean(mcc_scores)
avg_auc_roc = np.mean(auc_roc_scores)

# Print the average scores
print("Average Test Accuracy:", avg_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1 Score:", avg_f1)
print("Average MCC:", avg_mcc)
print("Average AUC-ROC:", avg_auc_roc)



Epoch 1/15
13/13 [==============================] - 5s 90ms/step - loss: 0.6875 - accuracy: 0.7225 - val_loss: 0.6830 - val_accuracy: 0.8700
Epoch 2/15
13/13 [==============================] - 0s 10ms/step - loss: 0.6827 - accuracy: 0.7975 - val_loss: 0.6789 - val_accuracy: 0.9100
Epoch 3/15
13/13 [==============================] - 0s 10ms/step - loss: 0.6777 - accuracy: 0.8675 - val_loss: 0.6746 - val_accuracy: 0.9100
Epoch 4/15
13/13 [==============================] - 0s 9ms/step - loss: 0.6735 - accuracy: 0.9000 - val_loss: 0.6701 - val_accuracy: 0.9000
Epoch 5/15
13/13 [==============================] - 0s 9ms/step - loss: 0.6692 - accuracy: 0.9150 - val_loss: 0.6655 - val_accuracy: 0.9100
Epoch 6/15
13/13 [==============================] - 0s 11ms/step - loss: 0.6640 - accuracy: 0.9050 - val_loss: 0.6605 - val_accuracy: 0.9200
Epoch 7/15
13/13 [==============================] - 0s 10ms/step - loss: 0.6593 - accuracy: 0.9150 - val_loss: 0.6553 - val_accuracy: 0.9200
Epoch 8/15
13/1

**LR=0.00001**

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score

# Assuming X and Y are your input and target data
# Define the number of folds
num_folds = 5

# Initialize lists to store the evaluation results
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
mcc_scores = []
auc_roc_scores = []



# Perform stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True)
for train_index, test_index in skf.split(X, Y):
    # Split the data into training and test sets for the current fold
    X_train, X_test = X[train_index], X[test_index]
    # X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    #start
    X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout

    model = Sequential()

# Add an LSTM layer with desired units and input shape
    model.add(LSTM(units=50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))  # Adding dropout for regularization

# Add another LSTM layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))

# Add a dense output layer for classification (adjust the number of units according to your task)
    model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
    custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train_reshaped, Y_train, epochs=15, batch_size=32, validation_data=(X_test_reshaped, Y_test))
    Y_pred = model.predict(X_test_reshaped)
    Y_pred_binary = np.round(Y_pred).flatten()

    # Calculate evaluation metrics
    accuracy = accuracy_score(Y_test, Y_pred_binary)
    precision = precision_score(Y_test, Y_pred_binary)
    recall = recall_score(Y_test, Y_pred_binary)
    f1 = f1_score(Y_test, Y_pred_binary)
    mcc = matthews_corrcoef(Y_test, Y_pred_binary)
    auc_roc = roc_auc_score(Y_test, Y_pred_binary)

    # Append the scores to the respective lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    mcc_scores.append(mcc)
    auc_roc_scores.append(auc_roc)

# Calculate the average scores
avg_accuracy = np.mean(accuracy_scores)
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_f1 = np.mean(f1_scores)
avg_mcc = np.mean(mcc_scores)
avg_auc_roc = np.mean(auc_roc_scores)

# Print the average scores
print("Average Test Accuracy:", avg_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1 Score:", avg_f1)
print("Average MCC:", avg_mcc)
print("Average AUC-ROC:", avg_auc_roc)



Epoch 1/15
13/13 [==============================] - 8s 184ms/step - loss: 0.6960 - accuracy: 0.4050 - val_loss: 0.6934 - val_accuracy: 0.4300
Epoch 2/15
13/13 [==============================] - 0s 10ms/step - loss: 0.6952 - accuracy: 0.4300 - val_loss: 0.6930 - val_accuracy: 0.4300
Epoch 3/15
13/13 [==============================] - 0s 12ms/step - loss: 0.6939 - accuracy: 0.4625 - val_loss: 0.6926 - val_accuracy: 0.4700
Epoch 4/15
13/13 [==============================] - 0s 10ms/step - loss: 0.6941 - accuracy: 0.4675 - val_loss: 0.6923 - val_accuracy: 0.4800
Epoch 5/15
13/13 [==============================] - 0s 12ms/step - loss: 0.6934 - accuracy: 0.4800 - val_loss: 0.6919 - val_accuracy: 0.5300
Epoch 6/15
13/13 [==============================] - 0s 11ms/step - loss: 0.6928 - accuracy: 0.5075 - val_loss: 0.6916 - val_accuracy: 0.5700
Epoch 7/15
13/13 [==============================] - 0s 10ms/step - loss: 0.6926 - accuracy: 0.5300 - val_loss: 0.6912 - val_accuracy: 0.5900
Epoch 8/15
1

**Nadam with LR=0.001**

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score

# Assuming X and Y are your input and target data
# Define the number of folds
num_folds = 5

# Initialize lists to store the evaluation results
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
mcc_scores = []
auc_roc_scores = []



# Perform stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True)
for train_index, test_index in skf.split(X, Y):
    # Split the data into training and test sets for the current fold
    X_train, X_test = X[train_index], X[test_index]
    # X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    #start
    X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout

    model = Sequential()

# Add an LSTM layer with desired units and input shape
    model.add(LSTM(units=50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))  # Adding dropout for regularization

# Add another LSTM layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))

# Add a dense output layer for classification (adjust the number of units according to your task)
    model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
    custom_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
    model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train_reshaped, Y_train, epochs=15, batch_size=32, validation_data=(X_test_reshaped, Y_test))
    Y_pred = model.predict(X_test_reshaped)
    Y_pred_binary = np.round(Y_pred).flatten()

    # Calculate evaluation metrics
    accuracy = accuracy_score(Y_test, Y_pred_binary)
    precision = precision_score(Y_test, Y_pred_binary)
    recall = recall_score(Y_test, Y_pred_binary)
    f1 = f1_score(Y_test, Y_pred_binary)
    mcc = matthews_corrcoef(Y_test, Y_pred_binary)
    auc_roc = roc_auc_score(Y_test, Y_pred_binary)

    # Append the scores to the respective lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    mcc_scores.append(mcc)
    auc_roc_scores.append(auc_roc)

# Calculate the average scores
avg_accuracy = np.mean(accuracy_scores)
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_f1 = np.mean(f1_scores)
avg_mcc = np.mean(mcc_scores)
avg_auc_roc = np.mean(auc_roc_scores)

# Print the average scores
print("Average Test Accuracy:", avg_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1 Score:", avg_f1)
print("Average MCC:", avg_mcc)
print("Average AUC-ROC:", avg_auc_roc)



Epoch 1/15
13/13 [==============================] - 7s 94ms/step - loss: 0.6639 - accuracy: 0.8150 - val_loss: 0.6249 - val_accuracy: 0.9100
Epoch 2/15
13/13 [==============================] - 0s 11ms/step - loss: 0.5802 - accuracy: 0.9275 - val_loss: 0.5334 - val_accuracy: 0.9200
Epoch 3/15
13/13 [==============================] - 0s 9ms/step - loss: 0.4814 - accuracy: 0.9375 - val_loss: 0.4359 - val_accuracy: 0.9200
Epoch 4/15
13/13 [==============================] - 0s 11ms/step - loss: 0.3824 - accuracy: 0.9400 - val_loss: 0.3506 - val_accuracy: 0.9100
Epoch 5/15
13/13 [==============================] - 0s 10ms/step - loss: 0.3027 - accuracy: 0.9450 - val_loss: 0.2867 - val_accuracy: 0.9100
Epoch 6/15
13/13 [==============================] - 0s 11ms/step - loss: 0.2414 - accuracy: 0.9450 - val_loss: 0.2440 - val_accuracy: 0.9100
Epoch 7/15
13/13 [==============================] - 0s 10ms/step - loss: 0.1988 - accuracy: 0.9500 - val_loss: 0.2130 - val_accuracy: 0.9300
Epoch 8/15
13/

**SGD**

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score

# Assuming X and Y are your input and target data
# Define the number of folds
num_folds = 5

# Initialize lists to store the evaluation results
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
mcc_scores = []
auc_roc_scores = []



# Perform stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True)
for train_index, test_index in skf.split(X, Y):
    # Split the data into training and test sets for the current fold
    X_train, X_test = X[train_index], X[test_index]
    # X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    #start
    X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout

    model = Sequential()

# Add an LSTM layer with desired units and input shape
    model.add(LSTM(units=50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))  # Adding dropout for regularization

# Add another LSTM layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))

# Add a dense output layer for classification (adjust the number of units according to your task)
    model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
    custom_optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train_reshaped, Y_train, epochs=15, batch_size=32, validation_data=(X_test_reshaped, Y_test))
    Y_pred = model.predict(X_test_reshaped)
    Y_pred_binary = np.round(Y_pred).flatten()

    # Calculate evaluation metrics
    accuracy = accuracy_score(Y_test, Y_pred_binary)
    precision = precision_score(Y_test, Y_pred_binary)
    recall = recall_score(Y_test, Y_pred_binary)
    f1 = f1_score(Y_test, Y_pred_binary)
    mcc = matthews_corrcoef(Y_test, Y_pred_binary)
    auc_roc = roc_auc_score(Y_test, Y_pred_binary)

    # Append the scores to the respective lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    mcc_scores.append(mcc)
    auc_roc_scores.append(auc_roc)

# Calculate the average scores
avg_accuracy = np.mean(accuracy_scores)
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)
avg_f1 = np.mean(f1_scores)
avg_mcc = np.mean(mcc_scores)
avg_auc_roc = np.mean(auc_roc_scores)

# Print the average scores
print("Average Test Accuracy:", avg_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1 Score:", avg_f1)
print("Average MCC:", avg_mcc)
print("Average AUC-ROC:", avg_auc_roc)



Epoch 1/15
13/13 [==============================] - 6s 99ms/step - loss: 0.6944 - accuracy: 0.4450 - val_loss: 0.6941 - val_accuracy: 0.3800
Epoch 2/15
13/13 [==============================] - 0s 11ms/step - loss: 0.6947 - accuracy: 0.4450 - val_loss: 0.6940 - val_accuracy: 0.3800
Epoch 3/15
13/13 [==============================] - 0s 11ms/step - loss: 0.6937 - accuracy: 0.4850 - val_loss: 0.6939 - val_accuracy: 0.3800
Epoch 4/15
13/13 [==============================] - 0s 11ms/step - loss: 0.6947 - accuracy: 0.4550 - val_loss: 0.6939 - val_accuracy: 0.3900
Epoch 5/15
13/13 [==============================] - 0s 11ms/step - loss: 0.6943 - accuracy: 0.4475 - val_loss: 0.6938 - val_accuracy: 0.4000
Epoch 6/15
13/13 [==============================] - 0s 10ms/step - loss: 0.6944 - accuracy: 0.4600 - val_loss: 0.6937 - val_accuracy: 0.4000
Epoch 7/15
13/13 [==============================] - 0s 11ms/step - loss: 0.6946 - accuracy: 0.4450 - val_loss: 0.6936 - val_accuracy: 0.4000
Epoch 8/15
13